<link rel="stylesheet" href="../../styles/theme_style.css">
<!--link rel="stylesheet" href="../../styles/header_style.css"-->
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">

<table width="100%">
    <tr>
        <td id="image_td" width="15%" class="header_image_color_4"><div id="image_img" class="header_image_4"></div></td>
        <td class="header_text">ACC Sensor - Unit Conversion </td>
    </tr>
</table>

<div id="flex-container">
    <div id="diff_level" class="flex-item">
        <strong>Difficulty Level:</strong>   <span class="fa fa-star checked"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
                                <span class="fa fa-star"></span>
    </div>
    <div id="tag" class="flex-item-tag">
        <span id="tag_list">
            <table id="tag_list_table">
                <tr>
                    <td class="shield_left">Tags</td>
                    <td class="shield_right" id="tags">pre-process&#9729;ACC&#9729;conversion</td> 
                </tr>
            </table>
        </span>
        <!-- [OR] Visit https://img.shields.io in order to create a tag badge-->
    </div>
</div>

The <strong><span class="color2">OpenSignals</span></strong> outputted file formats contain raw data, so each sample has a digital unit.

In scientific terms it is recommended the use of specific units, like electric tension (V) or electric current (A).
Each sensor that PLUX commercialise has a datasheet where a transfer function is mentioned for unit conversion be done.

The next lines are intended to explain how this conversion can be programmatically done.

In spite of the unit conversion procedure has some common steps applicable to all sensors, the current <strong><span class="color5">Jupyter Notebook</span></strong> is dedicated to the unit conversion procedure of signals acquired with Accelerometer sensor.

<p class="steps">1 - Importation of the needed packages</p>

In [ ]:
# biosignalsnotebooks Python package with useful functions that support and complement the available Notebooks
import biosignalsnotebooks as bsnb

# Function used for creating a numpy array, where a mathematical operation can be applied to each entry in an easy and automatic way. On the other side, linspace, here will be used for generation of a time-axis.
from numpy import array, linspace

<p class="steps">2 - Download of the sensor datasheet (from <a href="https://biosignalsplux.com/index.php/learn/documentation">https://biosignalsplux.com/index.php/learn/documentation</a>). In this case we are working with ACC, being our file located at <a href="https://plux.info/datasheets/ACC_Sensor_Datasheet.pdf">https://plux.info/datasheets/ACC_Sensor_Datasheet.pdf</a> </p>

In [ ]:
# Embedding of .pdf file
from IPython.display import IFrame
IFrame('../../images/pre-process/unit_conversion_ACC/ACC_Sensor_Datasheet.pdf#page=2', width="100%", height="350")

<p class="steps">3 - Extraction of the transfer function from the beginning of the second page</p> 

\begin{equation}
ACC{(g)} = \frac{ADC-{C_{min}}}{{C_{max}}-{C_{min}}}\times{2}-{1}
\end{equation}

<table width="100%">
    <tr>
        <td width="25%" style="text-align:left;vertical-align:top">$ACC{(g)}$ - ACC value in g-force (g)</td>
        <td width="25%" style="text-align:left;vertical-align:top">$ADC$ - Value sampled from the acquisition channel</td>
        <td width="25%" style="text-align:left;vertical-align:top">$C_{min}$ - Minimum calibration value</td>
        <td width="25%" style="text-align:left;vertical-align:top">$C_{max}$ - Maximum calibration value </td>

</table>

Quoting the description available on the sensor datasheet:

<i>This sensor requires a calibration to provide reliable measurements. The resulting calibration values ($C_{min}$ and $C_{max}$) which are needed for the transfer function are determined by performing a very slow 360º rotation of the sensor around each axis to force the accelerometer to cross the gravity-imposed −1g and 1g</i>

So, $C_{min}$ and $C_{max}$ are the minimum and maximum RAW values registered on each channel during the calibration procedure.

<p class="steps">4 - Loading of data stored in <strong><span class="color2">biosignalsnotebooks</span></strong> own signal library</p> 

In [ ]:
# Data loading
data, header = bsnb.load_signal("https://drive.google.com/open?id=1Tky7YNZ95VkFN3F2rxM6Fwf_x2YTMYmx", get_header=True)

In the following cell, some relevant information is stored inside variables. This relevant information includes the mac-address of the device, channel number and signal acquisition parameters such as resolution and sampling rate.

For a detailed explanation of how to access this info, the <a href="../Load/signal_loading_preparatory_steps.ipynb" target="_blank">"Signal Loading - Working with File Header"<img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a> Notebook should be consulted.

In [ ]:
ch_X= "CH1" # Channel (X-axis)
ch_Y = "CH2" # Channel (Y-axis)
ch_Z= "CH3"# Channel (Z-axis)
sr = 1000 # Sampling rate
resolution = 16 # Resolution (number of available bits)

For our specific case (16 bits acquisition), the calibration values ($C_{min}$ and $C_{max}$) are:

In [ ]:
c_min = 28000
c_max = 38000

Access to acquired signal samples and storage inside a new variable.

In [ ]:
signal_X = data[ch_X]
signal_Y = data[ch_Y]
signal_Z = data[ch_Z]

<p class="steps">5 - Final unit conversion (to <span class="color5">g</span>) by applying the transfer function sample by sample</p>

In [ ]:
signal_X_ = ((array(signal_X) - c_min) / (c_max - c_min)) * 2 - 1
signal_Y_= ((array(signal_Y) - c_min) / (c_max - c_min)) * 2 - 1
signal_Z_ = ((array(signal_Z) - c_min) / (c_max - c_min)) * 2 - 1

<p class="steps">6 - Time axis generation</p>

In [ ]:
time = bsnb.generate_time(signal_X_, sr)

Comparison between RAW and mV signal.

In [ ]:
bsnb.plot([time, time,time,time, time,time], [signal_X, signal_X_,signal_Y,signal_Y_, signal_Z,signal_Z_], y_axis_label=["Raw Data", "Acceleration (g)", "Raw Data", "Acceleration (g)","Raw Data", "Acceleration (g)"], title=["X Channel (Raw)", "X Channel (g)", "Y Channel (Raw)", "Y Channel (g)", "Z Channel(RAW)","Z Channel(g)"], grid_lines=3, grid_columns=2, grid_plot=True)

<strong><span class="color2">Similar Notebooks</span>, dedicated to the unit conversion of other sensors, are available in the following "conversion" section of <a href="../MainFiles/by_tag.ipynb">Notebooks Grouped by Tag page<img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a> </span></strong>

<span class="color6">**Auxiliary Code Segment (should not be replicated by the user)**</span>

In [ ]:
from biosignalsnotebooks.__notebook_support__ import css_style_apply
css_style_apply()

In [ ]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>